# R workshop: <font color=blue> Generating KEGG Pathway view </font>

#### MLBI@DKU

### __0. Install required R packages__

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
# BiocManager::install("biocLite")
BiocManager::install()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'BiocVersion'



In [2]:
install.packages("reticulate")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("org.Mm.eg.db")
BiocManager::install("biomaRt")
BiocManager::install("gageData")
BiocManager::install("gage")
BiocManager::install("pathview")
install.packages("filesstrings")
install.packages("anndata")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'org.Hs.eg.db'

also installing the dependencies ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘plogr’, ‘Biostrings’, ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘KEGGREST’, ‘AnnotationDbi’


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'org.Mm.eg.db'

'getOption("repos")' replaces Bioconductor 

### __1. Load libraries and data__

In [4]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(filesstrings))
suppressPackageStartupMessages(library(pathview))
suppressPackageStartupMessages(library(gage))
suppressPackageStartupMessages(library(gageData))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(org.Mm.eg.db))
suppressPackageStartupMessages(library(reticulate))
suppressPackageStartupMessages(library(anndata))

In [5]:
reticulate::py_require("gdown")
reticulate::py_require("anndata")

In [6]:
## Install and load "KEGGPathviewGen4SCODA" package
devtools::install_github("combio-dku/KEGGPathviewGen4SCODA")
suppressPackageStartupMessages(library(KEGGPathviewGen4SCODA))

Skipping 5 packages not available: org.Mm.eg.db, org.Hs.eg.db, gageData, gage, pathview



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpdvFR05/remotes1372e6bd80c/combio-dku-KEGGPathviewGen4SCODA-d8577e3/DESCRIPTION’ ... OK
* preparing ‘KEGGPathviewGen4SCODA’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘KEGGPathviewGen4SCODA_0.0.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [30]:
## Required to download SCODA-processed sample data
system('pip install mlbi-datasets --upgrade')
reticulate::py_require("mlbi_datasets")
mlbi <- import("mlbi_datasets")

### __2A. Download a preprocessed data__

In [32]:
adata_t <- mlbi$load_scoda_processed_sample_data( 'Breast' )
adata_t

AnnData object with n_obs × n_vars = 33742 × 22585
    obs: 'Patient', 'Description', 'Source', 'Condition', 'Menopause', 'Parity', 'Gender', 'geo_no', 'subtype', 'subtype_detail', 'tissue', 'condition', 'sample', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset', 'cnv_ref_ind', 'ploidy_score', 'ploidy_dec', 'ploidy_init_group', 'condition_for_deg', 'sample_ext_for_deg', 'celltype_for_deg', 'celltype_for_cci', 'tumor_origin_ind'
    var: 'variable_genes', 'chr', 'spot_no'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'HiCAT_summary', 'Pathways_DB', 'analysis_parameters', 'cnv', 'cnv_neighbors_info', 'inferploidy_summary', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_cnv', 'X_cnv_pca', 'X_pca', 'inferploidy_results'
    obsp: 'cnv_neighbor_graph_connectivity', 'cnv_neighbor_graph_distanc

### __2AB. Or, Load your scoda-processed data__

In [11]:
### Load data & extract cell-gene matrix as a data.frame (rownames: cell barcode, colnames: gene symbol)
file_h5ad <- 'BRCA_GSE161529_33K_v1_scoda_250731.h5ad'

adata_t <- read_h5ad(file_h5ad)
adata_t

AnnData object with n_obs × n_vars = 33742 × 22585
    obs: 'Patient', 'Description', 'Source', 'Condition', 'Menopause', 'Parity', 'Gender', 'geo_no', 'subtype', 'subtype_detail', 'tissue', 'condition', 'sample', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset', 'cnv_ref_ind', 'ploidy_score', 'ploidy_dec', 'ploidy_init_group', 'condition_for_deg', 'sample_ext_for_deg', 'celltype_for_deg', 'celltype_for_cci', 'tumor_origin_ind'
    var: 'variable_genes', 'chr', 'spot_no'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'HiCAT_summary', 'Pathways_DB', 'analysis_parameters', 'cnv', 'cnv_neighbors_info', 'inferploidy_summary', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_cnv', 'X_cnv_pca', 'X_pca', 'inferploidy_results'
    obsp: 'cnv_neighbor_graph_connectivity', 'cnv_neighbor_graph_distanc

### __3. Get KEGG pathway IDs and names__

In [12]:
df_pathways_map <- get_pathways_map( adata_t, min_overlap = 0.85 )

  |======================================================================| 100%
Converting Pathways DB .. done.        


Warning message in load(url("https://pathview.uncc.edu/data/khier.rda")):
“URL 'https://pathview.uncc.edu/data/khier.rda': Timeout of 60 seconds was reached”


Error in load(url("https://pathview.uncc.edu/data/khier.rda")) : 
  cannot open the connection to 'https://pathview.uncc.edu/data/khier.rda'


### __4. Generate KEGG pathviews of a single target__

In [13]:
## Check celltype list
target_list <- names(adata_t$uns[['DEG']])
target_list

[1] "Aneuploid Epithelial cell" "B cell"                   
 [3] "Diploid Epithelial cell"   "Endothelial cell"         
 [5] "Epithelial cell"           "Fibroblast"               
 [7] "ILC"                       "Macrophage"               
 [9] "Mast cell"                 "Plasma cell"              
[11] "Smooth muscle cell"        "T cell CD4+"              
[13] "T cell CD8+"

In [ ]:
## Check pathways list
df_pathways_map[,'pw_name']

In [28]:
## Select items to generate pathview
items <- c('Cell cycle', 'Pathways in cancer',
           'ErbB signaling pathway', 'Estrogen signaling pathway')
## Set it NULL to generate pathviews for all pathways

In [29]:
target_cell <- 'Epithelial cell'
dir_saved <- save_kegg_pathviews( adata_t, target_cell, df_pathways_map, pathways = items,
                                  deg.p.val.cutoff = 0.05, gsa.p.val.cutoff = 0.1,
                                  deg.key = 'DEG', gsa.key = 'GSA_up' )

Epithelial cell: Diploid_vs_others(3695), ER+_vs_others(3211), HER2+_vs_others(2942), Normal_vs_others(3932), TNBC_vs_others(3965)
  Diploid_vs_others: 5 intersection with 87 in DB -> 3 
  |======================================================================| 100%
  ER+_vs_others: 5 intersection with 139 in DB -> 4 
  |======================================================================| 100%
  HER2+_vs_others: 5 intersection with 111 in DB -> 3 
  |======================================================================| 100%
  Normal_vs_others: 5 intersection with 150 in DB -> 2 
  |======================================================================| 100%
  TNBC_vs_others: 5 intersection with 123 in DB -> 2 
  |======================================================================| 100%
Epithelial cell: 5/5 - 2/2 - Pathways in cancer                                  
